In [ ]:
#overfitting

In [ ]:
#dropout

In [ ]:
#class 함수:   

In [ ]:
#convolutional nn (CNN)

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision

In [14]:
train_data = torchvision.datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = torchvision.transforms.ToTensor(), 
    download = True,            
)
test_data = torchvision.datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = torchvision.transforms.ToTensor()
)

In [15]:
X=torch.flatten(train_data.data,start_dim=1).float()
y=train_data.targets
X=X/255 # 데이터를 0~1로 정규화

In [16]:
X_test=torch.flatten(test_data.data,start_dim=1).float()
y_test=test_data.targets
X_test=X_test/255 # 데이터를 0~1로 정규화

In [17]:
din, dh1, dh2, dout = 28*28,128,64,10
model = torch.nn.Sequential(
    torch.nn.Linear(din,dh1), # input - hidden1
    torch.nn.ReLU(),         # activation function of hidden layer 
    torch.nn.Linear(dh1,dh2), # input - hidden1
    torch.nn.ReLU(),         # activation function of hidden layer 
    torch.nn.Linear(dh2,dout), # hidden - out
    torch.nn.Softmax(dim=1)
)

# loss and optimizer
learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [34]:
nums_epoch = 500
def train_val(model, criterion, optimizer, nums_epoch, X, y, X_test, y_test):
    for epoch in range(nums_epoch):
        y_pred = model(X)
        loss = criterion(y_pred,y)
        optimizer.zero_grad() # zero_grad(): 미분값 초기화
        loss.backward() # backpropagation
        optimizer.step() # update weights and biases
        
        if(epoch%100==0):
            y_pred = model(X)
            prediction = y_pred.max(1)[1] # max indices
            corrects = (prediction == y)
            accuracy_train = corrects.sum().float() / float( y.size(0) )
           
            y_test_pred = model(X_test)
            test_prediction = y_test_pred.max(1)[1] # max indices
            test_corrects = (test_prediction == y_test)
            accuracy_test = test_corrects.sum().float() / float( y_test_pred.size(0) )       
            print(epoch, "loss=", accuracy_train.item(), "test=", accuracy_test.item())
#train 데이터로 학습된 상수를 test 데이터를 입력하여 overfitting 검사

In [35]:
train_val(model,criterion, optimizer, nums_epoch, X, y, X_test, y_test);

0 loss= 0.9551166892051697 test= 0.9467999935150146
100 loss= 0.9668999910354614 test= 0.9549999833106995
200 loss= 0.9737499952316284 test= 0.9578999876976013
300 loss= 0.9786499738693237 test= 0.9610999822616577
400 loss= 0.9819833040237427 test= 0.9638000130653381


In [52]:
#dropout 몇개의 layer 요소를 끈다

In [53]:
din, dh1, dh2, dout = 28*28,128,64,10
model_dropout = torch.nn.Sequential(
    torch.nn.Linear(din,dh1), # input - hidden1
    torch.nn.ReLU(),         # activation function of hidden layer 
    torch.nn.Dropout(p=0.5),
    
    torch.nn.Linear(dh1,dh2), # input - hidden1
    torch.nn.ReLU(),         # activation function of hidden layer 
    torch.nn.Dropout(p=0.5),
    
    torch.nn.Linear(dh2,dout), # hidden - out
    torch.nn.Softmax(dim=1)
)

In [54]:
learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_dropout.parameters(),lr=learning_rate)

In [55]:
train_val(model_dropout,criterion, optimizer, nums_epoch, X, y, X_test, y_test);

0 loss= 0.11375000327825546 test= 0.11620000004768372
100 loss= 0.8080000281333923 test= 0.8120999932289124
200 loss= 0.9046333432197571 test= 0.9059000015258789
300 loss= 0.9256666898727417 test= 0.9240999817848206
400 loss= 0.9367166757583618 test= 0.9301000237464905


#class 설명 :
기존에선 6만개의 예측값을 구하고 실제 레이블 사이의 로스함수를 구하여
w를 수정함 6만개를 한번에 집어넣음 
1.효율적이지않음, 2.input 사이즈나 데이터가 많다면 학습 불가
0~99번
100~199번
~
59900~60000번 등
개인이 일정비율로 쪼개어 로스를 구하고 학습시킴
6만개가 1사이클이 돌면 1에포크
batch라고 함

## BACTH

In [58]:
train_data = torchvision.datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = torchvision.transforms.ToTensor(), 
    download = True,            
) # train
test_data = torchvision.datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = torchvision.transforms.ToTensor()
) # validation

In [59]:
# mini batch
batch_size = 100
train_loader = torch.utils.data.DataLoader(
    dataset = train_data,
    batch_size = batch_size,
    shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    dataset = test_data,
    batch_size = batch_size,
    shuffle = False
)

In [60]:
for x,y in train_loader:
    print(y)

tensor([7, 5, 8, 4, 8, 0, 9, 3, 0, 4, 8, 0, 3, 5, 6, 0, 0, 1, 6, 3, 2, 5, 5, 2,
        6, 9, 5, 8, 3, 3, 0, 0, 1, 6, 3, 0, 6, 0, 1, 4, 3, 0, 6, 8, 2, 3, 7, 9,
        9, 7, 6, 7, 1, 0, 5, 3, 7, 4, 2, 3, 4, 7, 3, 9, 8, 5, 3, 0, 1, 0, 0, 7,
        1, 2, 9, 9, 8, 2, 2, 4, 7, 6, 5, 9, 8, 3, 2, 7, 3, 2, 8, 1, 7, 8, 1, 7,
        6, 6, 4, 5])
tensor([3, 3, 9, 0, 8, 1, 4, 6, 7, 4, 3, 8, 1, 6, 6, 2, 2, 6, 4, 7, 6, 8, 8, 6,
        9, 5, 5, 3, 8, 1, 4, 8, 1, 2, 8, 1, 0, 5, 2, 9, 9, 2, 6, 0, 8, 2, 6, 4,
        1, 0, 6, 4, 9, 1, 1, 8, 9, 3, 1, 4, 9, 5, 8, 0, 9, 5, 2, 4, 3, 6, 1, 0,
        7, 9, 5, 7, 8, 1, 5, 5, 1, 1, 7, 5, 9, 4, 9, 3, 3, 5, 6, 3, 3, 1, 8, 1,
        2, 3, 9, 7])
tensor([2, 9, 6, 4, 2, 7, 1, 5, 2, 3, 5, 3, 7, 2, 3, 6, 8, 5, 8, 9, 4, 0, 5, 1,
        4, 4, 5, 9, 1, 4, 1, 2, 4, 2, 4, 3, 5, 1, 3, 8, 6, 1, 1, 7, 1, 7, 2, 1,
        1, 4, 8, 5, 0, 6, 1, 1, 4, 8, 7, 0, 0, 8, 4, 2, 6, 4, 5, 6, 1, 6, 5, 0,
        1, 5, 1, 3, 0, 5, 2, 2, 5, 3, 2, 7, 7, 3, 8, 6, 1, 4, 2, 8, 0, 2, 2, 5

In [61]:
for x_1,y_1 in test_loader:
    print(y_1)

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9])
tensor([6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6,
        7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2,
        0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7,
        2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3,
        1, 6, 4, 2])
tensor([3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5, 7, 2, 2, 7,
        1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9, 8, 7, 2, 3, 0, 4,
        4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 6, 8, 5, 7, 7, 9, 1, 8, 1, 8, 0, 3, 0,
        1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2, 6, 4, 1, 5, 8, 2, 9, 2, 0, 4

## class

In [63]:
def adder(x,y):
    return x+y

In [64]:
adder(1,2)

3

In [75]:
class cal:
    def two(self,a,b):
        self.a = a
        self.b = b
    def add(self):
        f = self.a + self.b
        return f
    def notadd(self):
        f = self.a - self.b
        return f
    def mul(self):
        f = self.a * self.b
        return f

In [76]:
x = cal()
x.two(1,3)
y = cal()
y.two(4,5)
y.add()

9

In [77]:
x.add()

4

In [78]:
#class로 미리 함수를 정하면 변수를 다른 것으로 정하여도 model class가 변하지 않음

In [85]:
x.notadd(), y.mul()

(-2, 20)

In [88]:
class cal2:
    def __init__(self, a, b): #초기값
        self.a, self.b = a,b
    def add(self):
        f = self.a + self.b
        return f

In [89]:
y = cal2(2,3)
y.add()

5

In [90]:
"""
din, dh1, dh2, dout = 28*28,128,64,10
model = torch.nn.Sequential(
    torch.nn.Linear(din,dh1), # input - hidden1
    torch.nn.ReLU(),         # activation function of hidden layer 
    torch.nn.Linear(dh1,dh2), # input - hidden1
    torch.nn.ReLU(),         # activation function of hidden layer 
    torch.nn.Linear(dh2,dout), # hidden - out
    torch.nn.Softmax(dim=1)
)
"""

import torch.nn as nn 
# 위와 같은 모델을 클래스를 이용하여 작성
class net(nn.Module): # nn.Module 상속(inheritance)
    def __init__(self): # 초기값
        super(net, self).__init__() # nn.Module의 함수를 model에서 사용 
        self.fc1 = nn.Linear(784, 128) #fc = foly network 연결된 layer란 의미
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64,10)

    def forward(self, x):
        #x = self.flatten(x,start_dim=1)/255 # flatten
        x = x.view(-1, 784).float()
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        x = nn.functional.relu(x)
        x = self.fc3(x)
        x = nn.functional.softmax(x,dim=1)
        return x

In [91]:
model = net()

In [92]:
model

net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)